In [ ]:
!pip install dspy-ai pandas

In [ ]:
!pip install python-dotenv openai

In [ ]:

# Set environment variables directly in Colab
import os
os.environ['OPENROUTER_API_KEY'] = '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'
os.environ['MODEL'] = 'z-ai/glm-4.5-air:free'
os.environ['TEMPERATURE'] = '0.7'

In [ ]:

# Set environment variables directly in Colab
import os
from getpass import getpass

# Ask for OpenRouter API key securely
api_key = getpass('Enter your OpenRouter API key: ')

os.environ['OPENROUTER_API_KEY'] = api_key
os.environ['MODEL'] = 'z-ai/glm-4.5-air:free'
os.environ['TEMPERATURE'] = '0.3'

# Also save to .env file for persistence
with open('.env', 'w') as f:
    f.write(f"OPENROUTER_API_KEY={api_key}\n")
    f.write(f"MODEL=z-ai/glm-4.5-air:free\n")
    f.write(f"TEMPERATURE=0.3\n")

Enter your OpenRouter API key: ··········


In [ ]:
#Their are 2 methods. The below cell uses "Chain of Thought" and other cell below it uses "Predict". The first one is more sophisticted. play with temperature change.

In [ ]:

import os
import logging
from typing import List, Dict, Any, Optional
from dotenv import load_dotenv

# Load environment variables
load_dotenv('/content/.env')

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Import required libraries
import dspy
from dspy import Example, Predict, ChainOfThought

# API configuration
API_KEY = os.getenv("OPENROUTER_API_KEY") or ""
MODEL = "z-ai/glm-4.5-air:free"
TEMPERATURE = 0.7

# Check if API key is available
if not API_KEY:
    raise ValueError("OPENROUTER_API_KEY not found in environment variables")

# Set up DSPy to use GLM via OpenRouter
lm = dspy.LM(
    model="openai/" + MODEL,
    api_key=API_KEY,
    temperature=TEMPERATURE,
    base_url="https://openrouter.ai/api/v1"
)
dspy.settings.configure(lm=lm)
logger.info(f"Successfully configured DSPy with {MODEL}")

# Import validation functions
def validate_taste(taste: str) -> str:
    """Validate the artistic taste category."""
    valid_tastes = ['photorealistic', 'oil painting', 'anime', 'cyberpunk', 'watercolor', '3d render', 'flat illustration']
    if taste.lower() not in valid_tastes:
        raise ValueError(f"Invalid taste. Choose from: {', '.join(valid_tastes)}")
    return taste.lower()

def validate_input(user_input: str) -> str:
    """Validate the user input description."""
    if not user_input or len(user_input.strip()) < 3:
        raise ValueError("Please provide a description (at least 3 characters)")
    return user_input.strip()

# Define DSPy signatures
class InitialPromptOptimization(dspy.Signature):
    """Optimize user input into a detailed image prompt for the specified artistic style."""
    taste = dspy.InputField(desc="Artistic style (e.g., photorealistic, anime)")
    user_input = dspy.InputField(desc="Simple user description")
    optimized_prompt = dspy.OutputField(desc="Detailed optimized prompt")
    reasoning = dspy.OutputField(desc="Step-by-step reasoning for the optimization")

class PromptEnhancement(dspy.Signature):
    """Enhance an optimized prompt with technical details and artistic refinements."""
    base_prompt = dspy.InputField(desc="Already optimized prompt")
    taste = dspy.InputField(desc="Artistic style")
    enhanced_prompt = dspy.OutputField(desc="Enhanced prompt with technical details and artistic refinements")
    reasoning = dspy.OutputField(desc="Step-by-step reasoning for the enhancement")

# Define DSPy modules
class InitialPromptOptimizer(dspy.Module):
    def __init__(self):
        super().__init__()
        # Changed from Predict to ChainOfThought
        self.optimize = ChainOfThought(InitialPromptOptimization)

    def forward(self, taste: str, user_input: str):
        return self.optimize(taste=taste, user_input=user_input)

class PromptEnhancer(dspy.Module):
    def __init__(self):
        super().__init__()
        # Changed from Predict to ChainOfThought
        self.enhance = ChainOfThought(PromptEnhancement)

    def forward(self, base_prompt: str, taste: str):
        return self.enhance(base_prompt=base_prompt, taste=taste)

class FullPromptOptimizer(dspy.Module):
    def __init__(self):
        super().__init__()
        self.initial_optimizer = InitialPromptOptimizer()
        self.enhancer = PromptEnhancer()

    def forward(self, taste: str, user_input: str):
        # Step 1: Initial prompt optimization
        optimized_result = self.initial_optimizer(taste=taste, user_input=user_input)

        # Step 2: Enhance the optimized prompt
        enhanced_result = self.enhancer(
            base_prompt=optimized_result.optimized_prompt,
            taste=taste
        )

        # Return the results as attributes of a simple object
        # Note: ChainOfThought returns 'reasoning' not 'optimized_reasoning' and 'enhanced_reasoning'
        class OptimizationResult:
            def __init__(self, optimized_prompt, reasoning, enhanced_prompt, enhanced_reasoning):
                self.optimized_prompt = optimized_prompt
                self.reasoning = reasoning
                self.enhanced_prompt = enhanced_prompt
                self.enhanced_reasoning = enhanced_reasoning

        return OptimizationResult(
            optimized_prompt=optimized_result.optimized_prompt,
            reasoning=optimized_result.reasoning,
            enhanced_prompt=enhanced_result.enhanced_prompt,
            enhanced_reasoning=enhanced_result.reasoning
        )

# Prepare training data
def create_training_data() -> List[Dict[str, str]]:
    """Create training examples for DSPy."""
    return [
        {
            'taste': 'photorealistic',
            'user_input': 'a cat sitting on a windowsill',
            'optimized_prompt': 'Ultra realistic photograph of a ginger cat sitting on a sunlit windowsill, detailed fur texture, sharp focus, natural lighting, 85mm lens, shallow depth of field',
            'optimized_reasoning': 'The original description is simple. I enhanced it by adding details about the cat\'s color, the lighting conditions, camera equipment, and photography techniques.',
            'enhanced_prompt': 'Ultra realistic photograph of a ginger cat sitting on a sunlit windowsill, detailed fur texture, sharp focus, natural lighting, 85mm lens, shallow depth of field, 4K resolution, f/1.8 aperture, golden hour glow, cinematic composition',
            'enhanced_reasoning': 'I added technical photography details like resolution, aperture, and lighting to make the prompt more specific for photorealistic rendering.'
        },
        {
            'taste': 'oil painting',
            'user_input': 'a mountain landscape',
            'optimized_prompt': 'Oil painting of majestic mountain landscape at sunset, impressionist style, visible brushstrokes, warm golden hour light, textured canvas, rich color palette',
            'optimized_reasoning': 'I enhanced the basic mountain landscape by specifying it as an oil painting in impressionist style, with details about brushwork and lighting.',
            'enhanced_prompt': 'Oil painting of majestic mountain landscape at sunset, impressionist style, visible brushstrokes, warm golden hour light, textured canvas, rich color palette, canvas texture visible, Rembrandt lighting, thick impasto technique, 24"x36" aspect ratio',
            'enhanced_reasoning': 'I added oil painting specific techniques like impasto, canvas texture, and referenced Rembrandt lighting to enhance the artistic quality.'
        },
        {
            'taste': 'anime',
            'user_input': 'a futuristic city',
            'optimized_prompt': 'Anime style illustration of a futuristic cyberpunk cityscape at night, neon lights, flying vehicles, detailed architecture, vibrant colors, high contrast',
            'optimized_reasoning': 'I took the basic futuristic city concept and transformed it into an anime-style cyberpunk city with specific visual elements.',
            'enhanced_prompt': 'Anime style illustration of a futuristic cyberpunk cityscape at night, neon lights, flying vehicles, detailed architecture, vibrant colors, high contrast, Studio Ghibli influences, detailed backgrounds, cel-shaded rendering, 1080p resolution',
            'enhanced_reasoning': 'I added anime-specific rendering techniques like cel-shading, referenced Studio Ghibli style, and included resolution details.'
        },
        {
            'taste': 'cyberpunk',
            'user_input': 'a street market',
            'optimized_prompt': 'Cyberpunk street market at night, neon signs, diverse crowd, stalls selling futuristic tech, rain-slicked streets, Blade Runner aesthetic',
            'optimized_reasoning': 'I enhanced the basic street market with cyberpunk elements like neon lighting, futuristic technology, and rain-slicked streets.',
            'enhanced_prompt': 'Cyberpunk street market at night, neon signs, diverse crowd, stalls selling futuristic tech, rain-slicked streets, Blade Runner aesthetic, cinematic lighting, volumetric fog, reflections on wet pavement, ultra-detailed, 8K resolution',
            'enhanced_reasoning': 'I added cinematic elements like volumetric fog and reflections, and included technical details like 8K resolution for higher quality rendering.'
        },
        {
            'taste': 'watercolor',
            'user_input': 'a flower garden',
            'optimized_prompt': 'Watercolor painting of a vibrant flower garden, soft washes, fluid color transitions, transparent layers, loose brushwork',
            'optimized_reasoning': 'I enhanced the basic flower garden description by specifying watercolor techniques like washes and transparent layers.',
            'enhanced_prompt': 'Watercolor painting of a vibrant flower garden, soft washes, fluid color transitions, transparent layers, loose brushwork, wet-on-wet technique, visible paper texture, subtle bleeding of colors, delicate edges, pastel color palette, light and airy atmosphere',
            'enhanced_reasoning': 'I added specific watercolor techniques like wet-on-wet and references to the characteristic transparency and bleeding effects of watercolor painting.'
        },
        {
            'taste': '3d render',
            'user_input': 'a futuristic car',
            'optimized_prompt': '3D render of a futuristic car, sleek design, metallic surfaces, detailed model, clean lines',
            'optimized_reasoning': 'I transformed the basic car description into a 3D render by specifying modeling details and surface characteristics.',
            'enhanced_prompt': '3D render of a futuristic car, sleek design, metallic surfaces, detailed model, clean lines, UV mapping, polygonal structure, PBR materials, subsurface scattering, specular highlights, studio lighting, octane render, 8K resolution, detailed interior, aerodynamic details',
            'enhanced_reasoning': 'I added 3D-specific technical details like UV mapping, polygonal structure, and rendering engine specifications to create a professional 3D render.'
        },
        {
            'taste': 'flat illustration',
            'user_input': 'a civil engineer working on construction projects',
            'optimized_prompt': 'Flat illustration of a civil engineer working on construction projects for quality audit of high-rise towers, soft color palette, clean lines, contemporary art style',
            'optimized_reasoning': 'I enhanced the basic description by specifying it as a flat illustration with a clean, contemporary style and soft colors to match the aesthetic.',
            'enhanced_prompt': 'Flat illustration of a civil engineer working on construction projects for quality audit of high-rise towers, soft color palette, a poster by Tom Whalen, featured on behance, context art, behance hd, art on instagram, storybook illustration, Pro freelance, Illustration agency, Popular on Dribbble, soft shadows, no contrast, clean ultrasharp focus, premium vector, hand drawn people, timeless art, human illustration, freepik, flat colours, their faces are visible, show less details, clean lines and smooth curves, 2d flat illustration, contemporary art illustration, contemporary painting, use minimum props, limited Colors, use light grey color overlay for shadow, use light white color overlay for highlights, delicate art, whimsy and wonder, whimsical, by Alice Lee style, Wax crayon brushes procreate style, hand drawn',
            'enhanced_reasoning': 'I added specific flat illustration techniques and references to artists known for this style, along with details about color palettes, line work, and digital tools used to create this aesthetic.'
        }
    ]

# Simple data container
class TrainingData:
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        return Example(
            taste=item['taste'],
            user_input=item['user_input'],
            optimized_prompt=item['optimized_prompt'],
            optimized_reasoning=item['optimized_reasoning'],
            enhanced_prompt=item['enhanced_prompt'],
            enhanced_reasoning=item['enhanced_reasoning']
        ).with_inputs('taste', 'user_input')

# Quality metric for DSPy evaluation
def prompt_quality_metric(example, pred, trace=None) -> float:
    """Calculate quality score for the final enhanced prompt."""
    prompt = pred.enhanced_prompt

    # Quality indicators
    has_details = 'detailed' in prompt.lower()
    has_style = any(style in prompt.lower() for style in ['style', 'technique', 'aesthetic'])
    has_technical = any(tech in prompt.lower() for tech in ['lighting', 'focus', 'texture', 'composition', 'resolution', 'aperture'])
    has_mood = any(mood in prompt.lower() for mood in ['mood', 'atmosphere', 'ambiance', 'feeling'])

    # Check if it includes relevant technical details for the style
    has_photography_terms = True
    has_painting_terms = True
    has_anime_terms = True
    has_cyberpunk_terms = True
    has_watercolor_terms = True
    has_3d_terms = True
    has_flat_illustration_terms = True

    if example.taste == 'photorealistic':
        has_photography_terms = any(term in prompt.lower() for term in ['aperture', 'f/', 'shutter', 'depth', 'bokeh', 'photography', 'lens', 'camera'])
    elif example.taste == 'oil painting':
        has_painting_terms = any(term in prompt.lower() for term in ['brushstroke', 'impasto', 'canvas', 'palette', 'texture', 'oil', 'painting', 'brush'])
    elif example.taste == 'anime':
        has_anime_terms = any(term in prompt.lower() for term in ['anime', 'cel-shaded', 'manga', 'studio ghibli', 'japanese', 'cartoon', 'illustration'])
    elif example.taste == 'cyberpunk':
        has_cyberpunk_terms = any(term in prompt.lower() for term in ['neon', 'futuristic', 'blade runner', 'rain-slicked', 'volumetric fog', 'cybernetic', 'high-tech'])
    elif example.taste == 'watercolor':
        has_watercolor_terms = any(term in prompt.lower() for term in ['wet-on-wet', 'washes', 'transparency', 'flow', 'watercolor', 'aqueous', 'bleed'])
    elif example.taste == '3d render':
        has_3d_terms = any(term in prompt.lower() for term in ['3d', 'render', 'polygon', 'model', 'uv mapping', 'wireframe', 'mesh', 'texture', 'blender', 'maya'])
    elif example.taste == 'flat illustration':
        has_flat_illustration_terms = any(term in prompt.lower() for term in [
            'flat illustration', 'flat colors', 'flat design', 'flat vector', '2d flat',
            'clean lines', 'smooth curves', 'soft shadows', 'no contrast', 'clean ultrasharp focus',
            'limited colors', 'light grey overlay', 'light white overlay', 'delicate art',
            'whimsy', 'whimsical', 'hand drawn', 'procreate', 'wax crayon', 'storybook'
        ])

    # Calculate score
    style_specific_indicators = []

    if example.taste == 'photorealistic':
        style_specific_indicators = [has_photography_terms]
    elif example.taste == 'oil painting':
        style_specific_indicators = [has_painting_terms]
    elif example.taste == 'anime':
        style_specific_indicators = [has_anime_terms]
    elif example.taste == 'cyberpunk':
        style_specific_indicators = [has_cyberpunk_terms]
    elif example.taste == 'watercolor':
        style_specific_indicators = [has_watercolor_terms]
    elif example.taste == '3d render':
        style_specific_indicators = [has_3d_terms]
    elif example.taste == 'flat illustration':
        style_specific_indicators = [has_flat_illustration_terms]

    # Combine general indicators with style-specific ones
    indicators = [has_details, has_style, has_technical, has_mood] + style_specific_indicators
    return sum(indicators) / len(indicators)

# Main DSPy-powered prompt generator
class DSPyPromptGenerator:
    def __init__(self):
        # Create training data
        self.trainset = TrainingData(create_training_data())

        # Initialize the full optimizer
        self.optimizer = FullPromptOptimizer()

        # Try to use BootstrapFewShot if available
        try:
            from dspy.teleprompter import BootstrapFewShot
            teleprompter = BootstrapFewShot(metric=prompt_quality_metric)
            self.compiled_optimizer = teleprompter.compile(self.optimizer, trainset=self.trainset)
            logger.info("Using compiled optimizer with BootstrapFewShot")
        except ImportError:
            try:
                from dspy.teleprompters import BootstrapFewShot
                teleprompter = BootstrapFewShot(metric=prompt_quality_metric)
                self.compiled_optimizer = teleprompter.compile(self.optimizer, trainset=self.trainset)
                logger.info("Using compiled optimizer with BootstrapFewShot")
            except ImportError:
                logger.warning("Could not import BootstrapFewShot from teleprompter module")
                logger.info("Using non-compiled optimizer instead")
                self.compiled_optimizer = self.optimizer

        self.results = []

    def generate(self, taste: str, user_input: str) -> str:
      """Generate an enhanced image prompt using DSPy-optimized modules."""
      try:
          # Validate inputs
          taste = validate_taste(taste)
          user_input = validate_input(user_input)
          logger.info(f"Validated input: {taste} - {user_input}")

          # Use the compiled DSPy optimizer or fallback
          logger.info("Running DSPy-optimized prompt generation...")
          result = self.compiled_optimizer(taste=taste, user_input=user_input)

          optimized_prompt = result.optimized_prompt
          optimized_reasoning = result.reasoning  # This exists in ChainOfThought
          final_prompt = result.enhanced_prompt
          final_reasoning = result.enhanced_reasoning  # This exists in ChainOfThought

          # Calculate quality score
          # Create a simple object with the enhanced prompt for the metric function
          class SimplePrediction:
              def __init__(self, enhanced_prompt):
                  self.enhanced_prompt = enhanced_prompt

          quality_score = prompt_quality_metric(
              Example(taste=taste, user_input=user_input),
              SimplePrediction(final_prompt)
          )

          # Store result
          self.results.append({
              'taste': taste,
              'user_input': user_input,
              'optimized_prompt': optimized_prompt,
              'optimized_reasoning': optimized_reasoning,
              'final_prompt': final_prompt,
              'final_reasoning': final_reasoning,
              'quality_score': quality_score
          })

          logger.info(f"Generated prompt with quality score: {quality_score:.2f}")
          logger.info(f"Optimization reasoning: {optimized_reasoning}")
          logger.info(f"Enhancement reasoning: {final_reasoning}")
          return final_prompt

      except Exception as e:
          logger.error(f"Error in prompt generation: {e}")
          fallback_prompt = f"{taste} style image of {user_input}, detailed, high quality"
          logger.info(f"Using fallback prompt: {fallback_prompt}")

          # Also store fallback result
          self.results.append({
              'taste': taste,
              'user_input': user_input,
              'optimized_prompt': fallback_prompt,
              'optimized_reasoning': 'Fallback due to error',
              'final_prompt': fallback_prompt,
              'final_reasoning': 'Fallback due to error',
              'quality_score': 0.0
          })

          return fallback_prompt

    def get_results(self) -> List[Dict[str, Any]]:
        """Get all generated results."""
        return self.results

def main():
    """Main function using actual DSPy."""
    print("=== DSPy-Optimized Image Prompt Generation System ===\n")
    print("Using DSPy with:")
    print("1. Defined signatures for initial optimization and enhancement")
    print("2. Multi-stage pipeline (Initial Optimizer → Enhancer)")
    print("3. Chain-of-Thought reasoning for both optimization steps")

    try:
        from dspy.teleprompter import BootstrapFewShot
        print("4. BootstrapFewShot for automatic prompt optimization")
    except ImportError:
        try:
            from dspy.teleprompters import BootstrapFewShot
            print("4. BootstrapFewShot for automatic prompt optimization")
        except ImportError:
            print("4. Basic prompt optimization (teleprompter module not available)")

    print("5. Quality-based metrics for evaluation")
    print(f"6. GLM-4.5-Air as the backend language model\n")

    # Test examples
    examples = [
        {
            "taste": "flat illustration",
            "user_input": """ A oversized engineer using magnifying glass doing inspection of building construction site under construction buildings signages equipment cranes, soft color palette, a poster by Tom Whalen, featured on behance, context art, behance hd, art on instagram, storybook illustration, Pro freelance, Illustration agency, Popular on Dribbble, soft shadows, no contrast, clean ultrasharp focus, premium vector, hand drawn people, timeless art, human illustration, freepik, flat colours, their faces are visible, show less details, clean lines and smooth curves, 2d flat illustration, contemporary art illustration, contemporary painting, use minimum props, limited Colors, use light grey color overlay for shadow, use light white color overlay for highlights, delicate art, whimsy and wonder, whimsical, by Alice Lee style, Wax crayon brushes procreate style, hand drawn """        }
    ]

    # Initialize the DSPy prompt generator
    generator = DSPyPromptGenerator()

    for i, example in enumerate(examples, 1):
        print(f"Example {i}:")
        print(f"Taste: {example['taste']}")
        print(f"User Input: {example['user_input']}")

        try:
            # Generate using DSPy
            result = generator.generate(
                taste=example['taste'],
                user_input=example['user_input']
            )

            print(f"\nDSPy-Optimized Prompt:")
            print(f"{generator.results[-1]['optimized_prompt']}")

            print(f"\nOptimization Reasoning:")
            print(f"{generator.results[-1]['optimized_reasoning']}")

            print(f"\nDSPy-Enhanced Prompt (Final Output):")
            print(f"{result}")

            print(f"\nEnhancement Reasoning:")
            print(f"{generator.results[-1]['final_reasoning']}")

            print(f"\nQuality Score: {generator.results[-1]['quality_score']:.2f}")
            print("✅ SUCCESS: DSPy-optimized prompt generation completed!\n")

        except Exception as e:
            print(f"❌ FAILED: {e}\n")

        print("="*70)

if __name__ == "__main__":
    main()

=== DSPy-Optimized Image Prompt Generation System ===

Using DSPy with:
1. Defined signatures for initial optimization and enhancement
2. Multi-stage pipeline (Initial Optimizer → Enhancer)
3. Chain-of-Thought reasoning for both optimization steps
4. Basic prompt optimization (teleprompter module not available)
5. Quality-based metrics for evaluation
6. GLM-4.5-Air as the backend language model

Example 1:
Taste: flat illustration
User Input:  A oversized engineer using magnifying glass doing inspection of building construction site under construction buildings signages equipment cranes, soft color palette, a poster by Tom Whalen, featured on behance, context art, behance hd, art on instagram, storybook illustration, Pro freelance, Illustration agency, Popular on Dribbble, soft shadows, no contrast, clean ultrasharp focus, premium vector, hand drawn people, timeless art, human illustration, freepik, flat colours, their faces are visible, show less details, clean lines and smooth curves

In [ ]:
#!/usr/bin/env python3
"""
DSPy implementation for generating enhanced image generation prompts using GLM 4.5 Air
"""

import os
import logging
from typing import List, Dict, Any, Optional
from dotenv import load_dotenv

# Load environment variables from .env file if it exists
load_dotenv()

# Set environment variables directly (this will override .env if set above)
os.environ['OPENROUTER_API_KEY'] = os.getenv('OPENROUTER_API_KEY', '')
os.environ['MODEL'] = os.getenv('MODEL', 'z-ai/glm-4.5-air:free')
os.environ['TEMPERATURE'] = os.getenv('TEMPERATURE', '0.3')

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Import required DSPy libraries
import dspy
from dspy import Example, Predict, ChainOfThought

class DSPyImagePromptGenerator:
    """
    Pure DSPy implementation for generating enhanced image generation prompts
    Using GLM 4.5 Air model via OpenRouter
    """

    def __init__(self, model: str = None, api_key: str = None, temperature: float = 0.3):
        """
        Initialize DSPy image prompt generator with GLM 4.5 Air

        Args:
            model: Model name (will use GLM 4.5 Air by default)
            api_key: API key for OpenRouter
            temperature: Temperature setting
        """
        self.model = model or os.getenv('MODEL', 'z-ai/glm-4.5-air:free')
        self.api_key = api_key or os.getenv('OPENROUTER_API_KEY')
        self.temperature = float(temperature or os.getenv('TEMPERATURE', 0.3))

        if not self.api_key:
            raise ValueError("OPENROUTER_API_KEY not found in environment variables. Please run Cell 2 to set it.")

        logger.info(f"Initialized DSPy image prompt generator with model: {self.model}, temperature: {self.temperature}")

        # Configure DSPy with GLM 4.5 Air via OpenRouter
        self.lm = dspy.LM(
            model="openrouter/" + self.model,  # Prefix with "openrouter/" for OpenRouter provider
            api_key=self.api_key,
            temperature=self.temperature,
            base_url="https://openrouter.ai/api/v1"
        )
        dspy.settings.configure(lm=self.lm)

        # Create examples for few-shot learning
        self.examples = [
            Example(
                taste="photorealistic",
                user_input="a cat sitting on a windowsill",
                enhanced_prompt="""You are an expert image prompt engineer. Transform this image concept into a photorealistic AI image generation prompt.

**Input**:
- Image Concept: "a cat sitting on a windowsill"
- Quality Style: "photorealistic"

**Output**:
Ultra realistic photograph of a ginger cat sitting on a sunlit windowsill, detailed fur texture with individual hairs visible, sharp focus on the subject's eyes, natural lighting with soft shadows, 85mm lens with shallow depth of field, visible dust particles in the light beam, detailed wood grain on the windowsill, warm color temperature"""
            ),
            Example(
                taste="photorealistic",
                user_input="a mountain landscape at sunset",
                enhanced_prompt="""You are an expert image prompt engineer. Transform this image concept into a photorealistic AI image generation prompt.

**Input**:
- Image Concept: "a mountain landscape at sunset"
- Quality Style: "photorealistic"

**Output**:
Breathtaking photorealistic landscape of snow-capped mountains at golden hour, warm sunset colors reflecting on a serene lake, volumetric lighting with sun rays breaking through clouds, ultra high detail with visible rock textures and snow crystals, 8k resolution, deep depth of field with foreground elements to establish scale, atmospheric perspective on distant peaks"""
            )
        ]

        # Create a DSPy module for prompt enhancement
        self.module = self.create_enhancement_module()

    def create_enhancement_module(self):
        """Create a DSPy module for prompt enhancement"""
        class PromptEnhancement(dspy.Module):
            def __init__(self):
                super().__init__()
                self.enhance = ChainOfThought("taste, user_input -> enhanced_prompt")

            def forward(self, taste, user_input):
                return self.enhance(taste=taste, user_input=user_input)

        return PromptEnhancement()

    def generate_prompt(self, taste: str, user_input: str) -> str:
        """
        Generate an enhanced image prompt using DSPy optimization

        Args:
            taste: Style/quality (e.g., "photorealistic")
            user_input: Image description

        Returns:
            Enhanced image prompt string
        """
        try:
            logger.info(f"Generating enhanced prompt with taste: {taste}")

            # Format examples for the prompt
            examples_text = ""
            for example in self.examples:
                examples_text += f"Example:\n"
                examples_text += f"Taste: {example.taste}\n"
                examples_text += f"User Input: {example.user_input}\n"
                examples_text += f"Enhanced Prompt: {example.enhanced_prompt}\n\n"

            # Use the LM directly to avoid potential issues with module compilation
            with dspy.settings.context(lm=self.lm):
                # Create a prediction with the LM
                pred = self.lm(
                    prompt=f"""Generate an enhanced image generation prompt with the following details:

Taste: {taste}
User Input: {user_input}

Examples:
{examples_text}

Generate a detailed, enhanced image prompt that captures the essence of the input with the specified style, following the structure and specifications from the examples."""
                )

                # If the LM returns a string, use it directly
                if isinstance(pred, str):
                    enhanced_prompt = pred
                else:
                    # If the LM returns a more complex object, extract the text
                    enhanced_prompt = str(pred)

                logger.info("Enhanced prompt generated successfully")
                return enhanced_prompt.strip()

        except Exception as e:
            logger.error(f"Error generating enhanced prompt: {str(e)}")
            raise

if __name__ == "__main__":
    # Test the prompt enhancement
    print("🚀 Testing DSPy Image Prompt Enhancement with GLM 4.5 Air")
    print("=" * 50)

    try:
        # Initialize the prompt generator with GLM 4.5 Air
        generator = DSPyImagePromptGenerator()
        print("✅ DSPy image prompt generator initialized successfully")

        # Test with sample inputs
        taste = "photorealistic"
        user_input = "a cat sitting on a windowsill"

        print(f"\n🔍 Testing with taste: {taste}")
        print(f"User input: {user_input}")

        # Generate the enhanced prompt
        enhanced_prompt = generator.generate_prompt(taste, user_input)
        print(f"\n✅ Enhanced prompt generated successfully:")
        print("=" * 50)
        print(enhanced_prompt)
        print("=" * 50)

        print("\n✅ Prompt enhancement test completed successfully!")

    except Exception as e:
        print(f"❌ Error during testing: {str(e)}")
        logger.error(f"Prompt enhancement test failed: {str(e)}")

🚀 Testing DSPy Image Prompt Enhancement with GLM 4.5 Air
✅ DSPy image prompt generator initialized successfully

🔍 Testing with taste: photorealistic
User input: a cat sitting on a windowsill

✅ Enhanced prompt generated successfully:
['**Enhanced Prompt**:  \nYou are an expert image prompt engineer. Transform this image concept into a photorealistic AI image generation prompt.  \n\n**Input**:  \n- Image Concept: "a cat sitting on a windowsill"  \n- Quality Style: "photorealistic"  \n\n**Output**:  \nUltra realistic photograph of a sleek black cat perched on a weathered oak windowsill, bathed in soft morning light streaming through a half-open window, intricate details in the cat’s fur—visible undercoat and individual whiskers, sharp focus on the cat’s piercing green eyes with natural catchlights, shallow depth of field blurring the background slightly, visible dust motes dancing in the light beam, textured wood grain on the windowsill with minor paint chipping, warm golden hour glow c

In [ ]:

# Test with different inputs
if __name__ == "__main__":
    print("\n🎨 Testing with different image concepts")
    print("=" * 50)

    # Initialize generator
    generator = DSPyImagePromptGenerator()

    # Test with different inputs
    test_cases = [
        ("photorealistic", "front view welcome onboard banner in corporate office civil")
    ]

    for taste, user_input in test_cases:
        print(f"\n🔍 Testing with taste: {taste}")
        print(f"User input: {user_input}")

        try:
            enhanced_prompt = generator.generate_prompt(taste, user_input)
            print(f"\n✅ Enhanced prompt generated successfully:")
            print("=" * 50)
            print(enhanced_prompt)  # Changed: removed the [:500] truncation
            print("=" * 50)
        except Exception as e:
            print(f"❌ Error generating prompt: {str(e)}")


🎨 Testing with different image concepts

🔍 Testing with taste: photorealistic
User input: front view welcome onboard banner in corporate office civil

✅ Enhanced prompt generated successfully:
['\n**Enhanced Prompt**:  \nUltra realistic photograph of a front view "Welcome Onboard" banner in a modern corporate office setting, featuring a sleek vinyl banner with metallic lettering and subtle corporate branding, displayed on a minimalist wall backdrop. The office includes a polished reception desk with marble surface, ergonomic chairs, and floor-to-ceiling windows overlooking a city skyline. Natural daylight streams in, casting soft shadows and illuminating dust motes in the air. Shallow depth of field with the banner in sharp focus, while background elements like a potted fiddle-leaf fig and abstract art remain softly blurred. Ultra high detail showcasing fabric texture, wood grain on the desk, and metallic accents. 8k resolution, professional color grading with cool blue undertones and